# Running a powerflow given injection data

This notebook will outline how to run powerflow algorithms given a grid and injection snapshot. The example in this notebook includes an incomplete dataset, and some synthetic data and powerflow tuning are utilized to run a convergent powerflow. The specifics of the approach taken here are motivated, but not optimal! We encourage the reader to try their own ideas.

In [1]:
import pypowsybl as pp
import pandas as pd
from pathlib import Path
import numpy as np

grid = pp.network.load(file='recollement-auto-20210101-0000-enrichi.xiidm.bz2')
print('Grid model loaded')

grid.get_generators()

grid1= grid

Grid model loaded


This notebook uses injections that are in the .parquet format, for which we can use the pyarrow engine to read and convert them to a pandas dataframe. We can compare the below injected generator snapshot to the grid generator data above, noticing that much of the information is simply repeated, but that the critical values for `target_p` (the generator´s real power output) are included in the injection. These are necessary for us to be able to run a power flow.

In [2]:
#These snapshots are separated by element type, so we create a pandas dataframe for each of those we are interested in: the generators, loads and buses

df_inj_gen = pd.read_parquet("snapshot_gen_2021-01-01T00-00-00.parquet", engine="pyarrow")
df_inj_load = pd.read_parquet("snapshot_load_2021-01-01T00-00-00.parquet", engine="pyarrow")
df_inj_bus = pd.read_parquet("snapshot_bus_2021-01-01T00-00-00.parquet", engine="pyarrow")

#We can look into one of these snapshots to see what it contains
df_inj_gen

,datetime,id,name,energy_source,target_p,min_p,max_p,min_q,max_q,rated_s,...,i,voltage_level_id,bus_id,connected,substation_id,nominal_v,region_code,disaggregation_weight,online_capacity_total,actual_output_total
0,2021-01-01,.CTLO3GROUP.1,NaN,hydro,0.150,0.0,0.150,NaN,NaN,NaN,...,NaN,.CTLHP3,.CTLHP3_0,True,.CTLH,63.0,84,0.002712,55.315300,2667.0
1,2021-01-01,.CTLO3GROUP.2,NaN,hydro,0.150,0.0,0.150,NaN,NaN,NaN,...,NaN,.CTLHP3,.CTLHP3_1,True,.CTLH,63.0,84,0.002712,55.315300,2667.0
2,2021-01-01,ARGIAINF,NaN,solar,0.000,0.0,0.007,NaN,NaN,NaN,...,NaN,ARGIAP3,NaN,False,ARGIA,63.0,75,0.000000,18.229900,0.0
3,2021-01-01,ARGOEIN2,NaN,wind,0.000,0.0,0.000,0.0,0.0,NaN,...,NaN,ARGOEP4,ARGOEP4_0,True,ARGOE,90.0,32,0.000000,49.534698,372.0
4,2021-01-01,ARGOEIN3,NaN,wind,0.144,0.0,0.144,NaN,NaN,NaN,...,NaN,ARGOEP4,ARGOEP4_0,True,ARGOE,90.0,32,0.002907,49.534698,372.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5620,2021-01-01,YQUELING,NaN,solar,0.000,0.0,0.023,NaN,NaN,NaN,...,NaN,YQUELP4,NaN,False,YQUEL,90.0,28,0.000000,0.510000,0.0
5621,2021-01-01,YVETOINF,NaN,solar,0.000,0.0,0.004,NaN,NaN,NaN,...,NaN,YVETOP4,NaN,False,YVETO,90.0,28,0.000000,0.510000,0.0
5622,2021-01-01,YVETOING,NaN,solar,0.000,0.0,0.004,NaN,NaN,NaN,...,NaN,YVETOP4,NaN,False,YVETO,90.0,28,0.000000,0.510000,0.0
5623,2021-01-01,YZEURIN2,NaN,solar,0.000,0.0,0.044,NaN,NaN,NaN,...,NaN,YZEURP3,NaN,False,YZEUR,63.0,84,0.000000,5.722600,0.0


## Injecting the data

To use this data in the powerflow, we will need to change the `target_p` column in the grid to match those in the injection data. Noting that the indexing on the injection (pandas) dataset is via the index number as default, and we must change it to the `id` in order to correctly match the generators to their injected power. You will notice that there is a multiplication factor of 100 present; this is due to the `target_p`s of this generator injection dataset being in per-units for a base power of 100MW. This is not the case for the `p0`s in the load injection dataset, which are given in MW.


In [3]:
df_inj_gen['target_p'] =  df_inj_gen['target_p'] *100

df_gens = grid.get_generators(attributes=['target_p']) 
df_gens.update(df_inj_gen.set_index('id'))
grid.update_generators(df=df_gens)

In much the same way, we can set the P values for our loads, and proceed with V and Q injection. In the case of this dataset, however, we do not have the V or Q data (you may check!). Nonetheless, we will be able to run a powerflow if we can come up with some feasible synthetic data. Below, we use synthetic Q for our loads by setting the load Q according to the corresponding load P, with a power factor of 0.98. 

In [4]:
load_power_factor = 0.98 

df_inj_load['p0'] = df_inj_load['p0']
df_loads = grid.get_loads(attributes=['p0'])
df_loads.update(df_inj_load.set_index('id'))
grid.update_loads(df=df_loads)

df_inj_load['q0'] = df_inj_load['p0'] * np.sqrt( (1-load_power_factor**2)/load_power_factor)
df_loads = grid.get_loads(attributes=['q0'])
df_loads.update(df_inj_load.set_index('id'))
grid.update_loads(df=df_loads)



We must also use synthetic data for the generators. Generators can either be PV with a voltage regulator (and `voltage_regulator_on = True`) or PQ (with `voltage_regulator_on = False`). For the PQ generators we can take the same approach as for the loads, and for the PV, the nominal voltage (`nominal_v`) of the connected voltage level is a sensible choice.

In [5]:
gen_power_factor = 0.8

# PQ generators
df_inj_pqgens = df_inj_gen.loc[df_inj_gen['voltage_regulator_on'] == False]
df_inj_pqgens['target_q'] = df_inj_pqgens['target_p']* np.sqrt( (1-gen_power_factor**2)/gen_power_factor)
df_pqgens = grid.get_generators(attributes=['target_q']) 
df_pqgens.update(df_inj_pqgens.set_index('id'))
grid.update_generators(df=df_pqgens)

# PV generators
df_inj_pvgens = df_inj_gen.loc[df_inj_gen['voltage_regulator_on'] == True]

lookup_v = df_inj_bus.drop_duplicates(subset='voltage_level_id').set_index('voltage_level_id')['nominal_v'] # Get nominal_v for each voltage_level_id
df_inj_pvgens['target_v'] = df_inj_pvgens['voltage_level_id'].map(lookup_v)
df_pvgens = grid.get_generators(attributes=['target_v'])
df_pvgens.update(df_inj_pvgens.set_index('id'))
grid.update_generators(df=df_pvgens)

We can use the `.validate()` function on our grid to check if it is set up correctly such that we can run a powerflow.

In [6]:
grid.validate()

PyPowsyblError: Dangling line '.SOTEL62SOTEL': p0 is invalid

This error is telling us there is an issue with (at least one of) our dangling lines. Dangling lines are the interconnections at the periphery of our grid, connecting to buses outside of the RTE dataset. These can export (or import) active and reactive power from/to our grid, and so we must also inject this data into the grid. Below, we have decided to calculate the active power mismatch of the grid, and distribute this mismatch amongst the dangling lines, weighted by the inverse reactance of the line, motivated by the idea that lower impedance lines carry more power. We´ll also say that these dangling lines supply no reactive power.

In [ ]:
df_dlines = grid.get_dangling_lines()

mismatch = grid.get_generators()['target_p'].sum() - grid.get_loads()['p0'].sum()
print(f"\nMismatch to distribute across {len(df_dlines)} dangling lines: {mismatch:.2f} MW")

df_dlines['weight'] = 1.0 / df_dlines['x'].replace(0, np.nan) 
df_dlines['p0'] = mismatch * df_dlines['weight'] / df_dlines['weight'].sum()              
df_dlines['q0'] = 0.0

grid.update_dangling_lines(df=df_dlines[['p0', 'q0']])



Mismatch to distribute across 45 dangling lines: 21956.99 MW


Now we can check again to see if we have all the required input data for the powerflow:

In [ ]:
grid.validate()

<ValidationLevel.STEADY_STATE_HYPOTHESIS: 1>

This `STEADY_STATE_HYPOTHESIS` validation level is necessary for us to be able to run our powerflow.

## Running the powerflow

We run the powerflow as below, and have a look at the results

In [ ]:
results = pp.loadflow.run_ac(grid)
results

[ComponentResult(connected_component_num=0, synchronous_component_num=0, status=MAX_ITERATION_REACHED, status_text=Reached Newton-Raphson max iterations limit, iteration_count=16, reference_bus_id='CHAFFP7_0', slack_bus_results=[SlackBusResult(id='CHAFFP7_0', active_power_mismatch=33342.95481401203)], distributed_active_power=0.0)]

The `status` component of our results is telling us that the powerflow algorithm failed to converge. This is okay, as we still haven´t looked into the various ways we can choose to run the powerflow. We have a good degree of flexibility when it comes to this, having access to two providers:

In [ ]:
pp.loadflow.get_provider_names()
#pp.loadflow.get_default_provider() #Run to check which is in use

['DynaFlow', 'OpenLoadFlow']

We will use the default [OpenLoadFlow](https://powsybl.readthedocs.io/projects/powsybl-open-loadflow/en/latest/) provider, however [Dynawo](https://powsybl.readthedocs.io/projects/powsybl-dynawo/en/latest/) is another option. 

Now we can look at the parameters of our powerflow. The default ones are:

In [ ]:
pp.loadflow.Parameters()

Parameters(voltage_init_mode=UNIFORM_VALUES, transformer_voltage_control_on=False, use_reactive_limits=True, phase_shifter_regulation_on=False, twt_split_shunt_admittance=False, shunt_compensator_voltage_control_on=False, read_slack_bus=True, write_slack_bus=True, distributed_slack=True, balance_type=PROPORTIONAL_TO_GENERATION_P_MAX, dc_use_transformer_ratio=True, countries_to_balance=[], component_mode=<ComponentMode.MAIN_CONNECTED: 0>, hvdc_ac_emulation=True, dc_power_factor=1.0, provider_parameters={})

We can leave these as is, but let´s change them in an effort to get convergence. In the script below you will see various changes to PowSyBl loadflow [parameters](https://powsybl.readthedocs.io/projects/powsybl-core/en/stable/simulation/loadflow/configuration.html) which should help our powerflow to converge.  There are long lists of 'provider parameters' which offer increased flexibility, and can be found on the corresponding provider´s documentation ( [Open Loadflow](https://powsybl.readthedocs.io/projects/powsybl-open-loadflow/en/latest/), [Dynawo](https://powsybl.readthedocs.io/projects/powsybl-dynawo/en/latest/)). It´s important to note that these provider parameters must be used as below, with all keys and values given as a string.

In [ ]:
ac_params = pp.loadflow.Parameters(
    voltage_init_mode=pp.loadflow.VoltageInitMode.DC_VALUES, # Initialize voltages as 1pu and voltage angles using a DC powerflow
    distributed_slack=True, # The active power mismatch will be distributed over the network
    balance_type=pp.loadflow.BalanceType.PROPORTIONAL_TO_GENERATION_P, # The participation factors for slack distribution are calculated using 'target_p'
    transformer_voltage_control_on=False, # Transformer voltage regulating will not be allowed 
    shunt_compensator_voltage_control_on=False, # Shunt compensator voltage regulating will not be allowed 

    provider_parameters={ 
        'maxNewtonRaphsonIterations': '200',
        'maxOuterLoopIterations': '50',
        'stateVectorScalingMode': 'MAX_VOLTAGE_CHANGE',  # Limits voltage vector updates
        'maxVoltageChangeStateVectorScalingMaxDv': '0.05',  # Limits maximum voltage magnitude update amount
        'maxVoltageChangeStateVectorScalingMaxDphi': '0.08726',  # Limits maximum voltage angle update amount
        'maxSlackBusCount': '10', 
        'newtonRaphsonConvEpsPerEq': '1e-1', # Stopping criterion
        'slackBusPMaxMismatch': '500.0', # Slack power is considered to be distributed when below 500MW
    }
)

Now we´ll try to run the powerflow again, also producing a report alongside it:

In [ ]:
rep = pp.report.ReportNode()
results = pp.loadflow.run_ac(grid, parameters= ac_params, report_node= rep )
rep # If we wish to read the report

+ 
   + Load flow on network 'recollement-auto-20210101-0000-enrichi'
      + Network CC0 SC0
         7 generators were discarded from voltage control because not started
         17 generators have been discarded from voltage control because of a too small reactive range
         + Network info
            Network has 6404 buses and 9405 branches
            Network balance: active generation=84787.5248086825 MW, active load=82109.27629748559 MW, reactive generation=10589.658268868923 MVar, reactive load=12824.411425352097 MVar
            Angle reference bus: CHAFFP7_0
            Slack bus: CHAFFP7_0
            Slack bus: TAVELP7_0
            Slack bus: WARANP7_0
            Slack bus: TERRIP7_0
            Slack bus: VIGY P7_0
            Slack bus: BARNAP7_0
            Slack bus: CHESNP7_0
            Slack bus: ARGOEP7_0
            Slack bus: SSV.OP7_2
            Slack bus: TABARP7_0
         Slack bus active power (-2678.2485111975843 MW) distributed in 1 distribution iter

The loadflow converged! We can look at a few key pieces of information from `results` if we choose. The more interesting information, however, will come from inspecting our grid post-powerflow.

In [ ]:
print(f"Status: {results[0].status_text}")
print(f"Iteration count: {results[0].iteration_count}")
print(f"Active power mismatch: {results[0].slack_bus_results[0].active_power_mismatch:.4f} MW")

df_buses = grid.get_buses()
df_vlevels = grid.get_voltage_levels()
df_buses = df_buses.merge(df_vlevels[['nominal_v']], left_on='voltage_level_id', right_index=True)
df_buses['v_pu'] = df_buses['v_mag'] / df_buses['nominal_v']

print(f"\n--- Bus Voltage Results ---")
print(f"Total buses: {len(df_buses)}")
print(f"\nVoltage magnitude (kV):")
print(df_buses['v_mag'].describe().to_string())
print(f"\nVoltage magnitude (pu):")
print(df_buses['v_pu'].describe().to_string())

out_of_range = df_buses[(df_buses['v_pu'] < 0.9) | (df_buses['v_pu'] > 1.1)]
print(f"\nBuses outside 0.9-1.1 pu: {len(out_of_range)} / {len(df_buses)}")

Status: Converged
Iteration count: 219
Active power mismatch: 10.7794 MW

--- Bus Voltage Results ---
Total buses: 6467

Voltage magnitude (kV):
count    6366.000000
mean      112.720679
std        90.787988
min        14.362402
25%        59.690563
50%        63.154645
75%       149.214370
max       414.858389

Voltage magnitude (pu):
count    6366.000000
mean        0.971562
std         0.047715
min         0.532492
25%         0.946121
50%         0.976688
75%         0.999949
max         1.256519

Buses outside 0.9-1.1 pu: 438 / 6467
